In [ ]:
figure_folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_rna_properties'
folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
! mkdir $figure_folder

prefix = 'isoform_transcriptions'
transcribed_fasta = '{}/{}.fa'.format(folder, prefix)
rnafold_results = transcribed_fasta.replace('.fa', '_rnafold.txt')

import re
from Bio import SeqIO
import forgi.graph.bulge_graph as fgb
import pandas as pd
from collections import Counter

acronym_to_full = {'f': 'five_prime', 't': 'three_prime', 's': 'stem', 'i': 'interior_loop', 'm': 'multiloop', 'h': 'hairpin'}

seriess = []

for i, record in enumerate(SeqIO.parse(rnafold_results, 'fasta')):
    if i+1 % 1000 == 0:
        print '{} sequences read'.format(i+1)
    event_name, isoform = record.id.split('|')
#     print record.seq
    s = str(record.seq).lstrip('ACGUT')
    search = re.search('([\+.(+)+]+)(\(-?\d+\.\d+\))', s)
    structure, mfe = search.groups()
    mfe = float(mfe.strip('()'))
    bg = fgb.BulgeGraph(dotbracket_str=structure)
    element_counts = pd.Series(Counter(bg.to_element_string()))
    element_counts = element_counts.rename(acronym_to_full)
    element_percentage = element_counts/element_counts.sum()
    element_percentage.index = element_percentage.index.map(lambda x: x + '_percentage')
    element_percentage['double_stranded_percentage'] = sum(1 for _ in structure if _ != '.')/float(len(s))
    element_percentage['minimum_free_energy'] = mfe
    element_percentage.index = element_percentage.index.map(lambda x: 'rnafold_'+ x)
    element_percentage['event_id'] = event_name
    element_percentage['isoform'] = isoform

    seriess.append(element_percentage)


mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_rna_properties': File exists


In [31]:
structure_df = pd.DataFrame(seriess)
structure_df.head()

,event_id,isoform,rnafold_double_stranded_percentage,rnafold_five_prime_percentage,rnafold_hairpin_percentage,rnafold_interior_loop_percentage,rnafold_minimum_free_energy,rnafold_multiloop_percentage,rnafold_stem_percentage,rnafold_three_prime_percentage
0,exon:chr10:100190328-100190427:-@exon:chr10:10...,isoform2,0.555957,0.037175,0.104089,0.223048,-95.9,0.052045,0.572491,0.011152
1,exon:chr10:100190328-100190427:-@exon:chr10:10...,isoform1,0.483146,0.058824,0.105882,0.205882,-47.1,0.105882,0.505882,0.017647
2,exon:chr10:100193697-100193848:-@exon:chr10:10...,isoform2,0.644550,NaN,0.118644,0.096852,-183.1,0.111380,0.658596,0.014528
3,exon:chr10:100193697-100193848:-@exon:chr10:10...,isoform1,0.628352,NaN,0.150794,0.095238,-108.7,0.079365,0.650794,0.023810
4,exon:chr10:100195392-100195529:-@exon:chr10:10...,isoform2,0.651584,NaN,0.096998,0.154734,-168.8,0.078522,0.665127,0.004619


In [32]:
structure_df.isnull().any()

event_id                              False
isoform                               False
rnafold_double_stranded_percentage    False
rnafold_five_prime_percentage          True
rnafold_hairpin_percentage             True
rnafold_interior_loop_percentage       True
rnafold_minimum_free_energy           False
rnafold_multiloop_percentage           True
rnafold_stem_percentage                True
rnafold_three_prime_percentage         True
dtype: bool

In [33]:
structure_df = structure_df.fillna(0)
structure_df.head()

,event_id,isoform,rnafold_double_stranded_percentage,rnafold_five_prime_percentage,rnafold_hairpin_percentage,rnafold_interior_loop_percentage,rnafold_minimum_free_energy,rnafold_multiloop_percentage,rnafold_stem_percentage,rnafold_three_prime_percentage
0,exon:chr10:100190328-100190427:-@exon:chr10:10...,isoform2,0.555957,0.037175,0.104089,0.223048,-95.9,0.052045,0.572491,0.011152
1,exon:chr10:100190328-100190427:-@exon:chr10:10...,isoform1,0.483146,0.058824,0.105882,0.205882,-47.1,0.105882,0.505882,0.017647
2,exon:chr10:100193697-100193848:-@exon:chr10:10...,isoform2,0.644550,0.000000,0.118644,0.096852,-183.1,0.111380,0.658596,0.014528
3,exon:chr10:100193697-100193848:-@exon:chr10:10...,isoform1,0.628352,0.000000,0.150794,0.095238,-108.7,0.079365,0.650794,0.023810
4,exon:chr10:100195392-100195529:-@exon:chr10:10...,isoform2,0.651584,0.000000,0.096998,0.154734,-168.8,0.078522,0.665127,0.004619


In [34]:
structure_df = structure_df.set_index(['event_id', 'isoform'])

In [37]:
structure_df_unstacked = structure_df.unstack()
structure_df_unstacked.head()

rnafold_double_stranded_percentage  \
isoform                                                                      isoform1   
event_id                                                                                
exon:chr10:100190328-100190427:-@exon:chr10:100...                           0.483146   
exon:chr10:100193697-100193848:-@exon:chr10:100...                           0.628352   
exon:chr10:100195392-100195529:-@exon:chr10:100...                           0.655518   
exon:chr10:101165513-101165617:-@exon:chr10:101...                           0.587814   
exon:chr10:101419263-101419345:+@exon:chr10:101...                           0.591241   

                                                              \
isoform                                             isoform2   
event_id                                                       
exon:chr10:100190328-100190427:-@exon:chr10:100...  0.555957   
exon:chr10:100193697-100193848:-@exon:chr10:100...  0.644550   
exon:chr10:100195392-100195529:-@exon:chr10:100...  0.651584   
exon:chr10:101165513-101165617:-@exon:chr10:101...  0.617169   
exon:chr10:101419263-101419345:+@exon:chr10:101...  0.550265   

                                                   rnafold_five_prime_percentage  \
isoform                                                                 isoform1   
event_id                                                                           
exon:chr10:100190328-100190427:-@exon:chr10:100...                      0.058824   
exon:chr10:100193697-100193848:-@exon:chr10:100...                      0.000000   
exon:chr10:100195392-100195529:-@exon:chr10:100...                      0.000000   
exon:chr10:101165513-101165617:-@exon:chr10:101...                      0.007380   
exon:chr10:101419263-101419345:+@exon:chr10:101...                      0.007519   

                                                              \
isoform                                             isoform2   
event_id                                                       
exon:chr10:100190328-100190427:-@exon:chr10:100...  0.037175   
exon:chr10:100193697-100193848:-@exon:chr10:100...  0.000000   
exon:chr10:100195392-100195529:-@exon:chr10:100...  0.000000   
exon:chr10:101165513-101165617:-@exon:chr10:101...  0.004739   
exon:chr10:101419263-101419345:+@exon:chr10:101...  0.005420   

                                                   rnafold_hairpin_percentage  \
isoform                                                              isoform1   
event_id                                                                        
exon:chr10:100190328-100190427:-@exon:chr10:100...                   0.105882   
exon:chr10:100193697-100193848:-@exon:chr10:100...                   0.150794   
exon:chr10:100195392-100195529:-@exon:chr10:100...                   0.120690   
exon:chr10:101165513-101165617:-@exon:chr10:101...                   0.180812   
exon:chr10:101419263-101419345:+@exon:chr10:101...                   0.093985   

                                                              \
isoform                                             isoform2   
event_id                                                       
exon:chr10:100190328-100190427:-@exon:chr10:100...  0.104089   
exon:chr10:100193697-100193848:-@exon:chr10:100...  0.118644   
exon:chr10:100195392-100195529:-@exon:chr10:100...  0.096998   
exon:chr10:101165513-101165617:-@exon:chr10:101...  0.106635   
exon:chr10:101419263-101419345:+@exon:chr10:101...  0.192412   

                                                   rnafold_interior_loop_percentage  \
isoform                                                                    isoform1   
event_id                                                                              
exon:chr10:100190328-100190427:-@exon:chr10:100...                         0.205882   
exon:chr10:100193697-100193848:-@exon:chr10:100...                         0.095238   
exon:chr10:100195392-100195529:-@exon:chr1

### Rename columns

In [42]:
structure_df_unstacked.columns = map(lambda x: '{}_{}'.format(x[1], x[0]), structure_df_unstacked.columns.values)
structure_df_unstacked.head()

,isoform1_rnafold_double_stranded_percentage,isoform2_rnafold_double_stranded_percentage,isoform1_rnafold_five_prime_percentage,isoform2_rnafold_five_prime_percentage,isoform1_rnafold_hairpin_percentage,isoform2_rnafold_hairpin_percentage,isoform1_rnafold_interior_loop_percentage,isoform2_rnafold_interior_loop_percentage,isoform1_rnafold_minimum_free_energy,isoform2_rnafold_minimum_free_energy,isoform1_rnafold_multiloop_percentage,isoform2_rnafold_multiloop_percentage,isoform1_rnafold_stem_percentage,isoform2_rnafold_stem_percentage,isoform1_rnafold_three_prime_percentage,isoform2_rnafold_three_prime_percentage
event_id,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,0.483146,0.555957,0.058824,0.037175,0.105882,0.104089,0.205882,0.223048,-47.1,-95.9,0.105882,0.052045,0.505882,0.572491,0.017647,0.011152
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,0.628352,0.644550,0.000000,0.000000,0.150794,0.118644,0.095238,0.096852,-108.7,-183.1,0.079365,0.111380,0.650794,0.658596,0.023810,0.014528
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,0.655518,0.651584,0.000000,0.000000,0.120690,0.096998,0.182759,0.154734,-116.1,-168.8,0.013793,0.078522,0.675862,0.665127,0.006897,0.004619
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,0.587814,0.617169,0.007380,0.004739,0.180812,0.106635,0.092251,0.191943,-79.6,-131.2,0.103321,0.063981,0.605166,0.630332,0.011070,0.002370
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,0.591241,0.550265,0.007519,0.005420,0.093985,0.192412,0.105263,0.073171,-86.2,-122.2,0.142857,0.135501,0.609023,0.563686,0.041353,0.029810


In [43]:
folder

'/projects/ps-yeolab/obotvinnik/singlecell_pnms'

In [44]:
splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(folder), index_col=0)
print splicing_feature_data.shape
splicing_feature_data.head()

(33693, 67)


/home/obotvinnik/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (11,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,...,exon2_phastcons_placental_mammal,isoform_domain_effects,isoform1_pfam_accession,isoform2_pfam_accession,isoform1_pfam_name,isoform2_pfam_name,isoform1_pfam_go_name,isoform2_pfam_go_name,isoform1_pfam_go_id,isoform2_pfam_go_id
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,...,0.291112,No annotated reading frame --> Protein +/- domain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,...,0.803519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,...,0.839394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,...,0.978780,No annotated reading frame --> Protein +/- domain,NaN,PF00155.16[1],NaN,Aminotran_1_2[1],NaN,"(pyridoxal phosphate binding)[1],(biosynthetic...",NaN,"GO:0030170[1],GO:0009058[1]"
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,...,0.934804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
splicing_feature_data = splicing_feature_data.join(structure_df_unstacked)

In [47]:
splicing_feature_data.head()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,...,isoform1_rnafold_interior_loop_percentage,isoform2_rnafold_interior_loop_percentage,isoform1_rnafold_minimum_free_energy,isoform2_rnafold_minimum_free_energy,isoform1_rnafold_multiloop_percentage,isoform2_rnafold_multiloop_percentage,isoform1_rnafold_stem_percentage,isoform2_rnafold_stem_percentage,isoform1_rnafold_three_prime_percentage,isoform2_rnafold_three_prime_percentage
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,...,0.205882,0.223048,-47.1,-95.9,0.105882,0.052045,0.505882,0.572491,0.017647,0.011152
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,...,0.095238,0.096852,-108.7,-183.1,0.079365,0.111380,0.650794,0.658596,0.023810,0.014528
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,...,0.182759,0.154734,-116.1,-168.8,0.013793,0.078522,0.675862,0.665127,0.006897,0.004619
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,...,0.092251,0.191943,-79.6,-131.2,0.103321,0.063981,0.605166,0.630332,0.011070,0.002370
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,...,0.105263,0.073171,-86.2,-122.2,0.142857,0.135501,0.609023,0.563686,0.041353,0.029810


In [48]:
splicing_feature_data.shape

(33693, 83)

In [49]:
splicing_feature_data.to_csv('{}/splicing_feature_data.csv'.format(folder))